# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [46]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [47]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [48]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local', 'uk_food']


In [49]:
# assign the uk_food database to a variable name
uk_food_db = mongo['uk_food']

In [50]:
# review the collections in our new database
print(uk_food_db.list_collection_names())

['establishments']


In [51]:
# review a document in the establishments collection
uk_food_db.establishments.find_one()

{'_id': ObjectId('66662babfecb2dfec6f90e3f'),
 'FHRSID': 254719,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT15 6DY',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2022-03-24T00:00:00',
 'LocalAuthorityCode': '182',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'scores': {'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.387974', 'latitude': '51.152225'},
 'RightToReply': '',
 'Distance': 4587.347174863443,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCount': 0,
  'returncode

In [52]:
# assign the collection to a variable
establishments = uk_food_db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [53]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [54]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

InsertOneResult(ObjectId('66662bf343f14a12542e08a9'), acknowledged=True)

In [55]:
# Check that the new restaurant was inserted
new_restaurant_find = list(establishments.find({"BusinessName":"Penang Flavours"}))
# Check to see that we only have the one new entry
print(len(new_restaurant_find))
# Display
pprint(new_restaurant_find)

1
[{'AddressLine1': 'Penang Flavours',
  'AddressLine2': '146A Plumstead Rd',
  'AddressLine3': 'London',
  'AddressLine4': '',
  'BusinessName': 'Penang Flavours',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': '',
  'Distance': 4623.972328074718,
  'LocalAuthorityCode': '511',
  'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
  'LocalAuthorityName': 'Greenwich',
  'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
  'NewRatingPending': True,
  'Phone': '',
  'PostCode': 'SE18 7DY',
  'RightToReply': '',
  'SchemeType': 'FHRS',
  '_id': ObjectId('66662bf343f14a12542e08a9'),
  'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
  'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}]


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [56]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
BusinessTypeID_find = list(establishments.find({"BusinessType":"Restaurant/Cafe/Canteen"},{"BusinessTypeID":1,"BusinessType":1}))
pprint(BusinessTypeID_find[0])

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('66662babfecb2dfec6f90e3f')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [57]:
# Update the new restaurant with the correct BusinessTypeID
verify_update = establishments.update_one(new_restaurant,{"$set":{"BusinessTypeID":1}})
print(verify_update)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)


In [65]:
# Confirm that the new restaurant was updated
update_verify = list(establishments.find({"BusinessName":"Penang Flavours"},{"BusinessTypeID":1, "BusinessName":1}))

print(f'Count of records changed: {len(update_verify)}')
for uv in update_verify:
    pprint(uv)

Count of records changed: 1
{'BusinessName': 'Penang Flavours',
 'BusinessTypeID': 1,
 '_id': ObjectId('66662bf343f14a12542e08a9')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [70]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName":{"$regex":"Dover"}})
print(dover_count)

994


In [71]:
# Delete all documents where LocalAuthorityName is "Dover"
dover_delete = establishments.delete_many({"LocalAuthorityName":{"$regex":"Dover"}})
print(dover_delete)

DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)


In [72]:
# Check if any remaining documents include Dover
dover_count = establishments.count_documents({"LocalAuthorityName":{"$regex":"Dover"}})
print(dover_count)

0


In [73]:
# Check that other documents remain with 'find_one'
pprint(establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66662babfecb2dfec6f91126'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [74]:
# Change the data type from String to Decimal for longitude and latitude
lon_lat_update = establishments.update_many({}, [{'$set': {'geocode.longitude': {'$toDouble': '$geocode.longitude'}, 'geocode.latitude': {'$toDouble': '$geocode.latitude'}}}])
print(lon_lat_update)

UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)


Use `update_many` to convert `RatingValue` to integer numbers.

In [75]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
update_rslts = establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])
print(update_rslts)

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)


In [76]:
# Change the data type from String to Integer for RatingValue
ratingvalue_update = establishments.update_many({}, [{'$set': {'RatingValue': {'$toInt': '$RatingValue'}}}])
print(ratingvalue_update)

UpdateResult({'n': 38786, 'nModified': 34695, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)


In [106]:
# Check that the coordinates and rating value are now numbers
rating_changes_int_count = establishments.count_documents({"RatingValue":{"$gte":0}})
rating_changes_str_count = establishments.count_documents({"RatingValue":{"$gte":'0'}})
rating_changes_none_count = establishments.count_documents({"RatingValue":{"$eq":None}})

print(f'{rating_changes_int_count} ratings stored as integers \n \
{rating_changes_str_count} ratings stored as strings \n \
{rating_changes_none_count} empty ratings \n')

coordinates_num_count = establishments.count_documents({"geocode.longitude":{"$gte":0}})
coordinate_str_count = establishments.count_documents({"geocode.latitude":{"$gte":'0'}})

print(f'{coordinates_num_count} geocodes stored as integers \n {coordinate_str_count} geocodes stored as strings')

34694 ratings stored as integers 
 0 ratings stored as strings 
 4092 empty ratings 

38786 geocodes stored as integers 
 0 geocodes stored as strings
